# Integrated Coastal Risk Framework - Quick Start

This notebook demonstrates the basic workflow for coastal risk assessment using the integrated framework.

## Workflow Overview:

1. Load and process hazard data
2. Build Bayesian network for compound hazards
3. Generate scenario ensemble
4. Run agent-based vulnerability model
5. Perform uncertainty analysis
6. Identify robust adaptation strategies

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import framework modules
from src.data_processing import CoastalDataProcessor
from src.bayesian_network import CompoundHazardNetwork
from src.agent_based_model import VulnerabilityModel
from src.uncertainty import DeepUncertaintyAnalysis

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Data Processing

Load historical hazard data for our target city (Lagos, Nigeria)

In [ ]:
# Initialize data processor
processor = CoastalDataProcessor(city='Lagos', data_dir='../data/raw')

# Load hazard data
hazard_data = processor.load_hazard_data(
    start_year=2000,
    end_year=2023,
    variables=['sea_level', 'storm_surge', 'precipitation', 'wave_height']
)

print(f"Loaded {len(hazard_data)} days of data")
print(f"Data quality score: {processor.data_quality_score:.2f}")
hazard_data.head()

In [ ]:
# Visualize historical trends
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

hazard_data.plot(x='date', y='sea_level', ax=axes[0, 0], title='Sea Level')
hazard_data.plot(x='date', y='storm_surge', ax=axes[0, 1], title='Storm Surge')
hazard_data.plot(x='date', y='precipitation', ax=axes[1, 0], title='Precipitation')
hazard_data.plot(x='date', y='wave_height', ax=axes[1, 1], title='Wave Height')

plt.tight_layout()
plt.show()

In [ ]:
# Load vulnerability data
vulnerability_data = processor.load_vulnerability_data()
print(f"\nLoaded vulnerability data for {len(vulnerability_data)} zones")
vulnerability_data.head()

## 2. Bayesian Network Construction

Build a Bayesian network to model dependencies between hazards

In [ ]:
# Create Bayesian network
hazard_types = ['storm_surge', 'sea_level', 'precipitation']

bn = CompoundHazardNetwork(
    hazard_types=hazard_types,
    city='Lagos',
    use_hierarchical=True
)

# Visualize network structure
bn.visualize_network()

In [ ]:
# Fit network to data
print("Fitting Bayesian network... (this may take a few minutes)")

bn.fit(
    data=hazard_data,
    n_samples=1000,  # Reduced for quick demo
    tune=500,
    chains=2
)

In [ ]:
# View parameter estimates
summary = bn.get_summary_statistics()
print("\nParameter Estimates:")
summary

## 3. Scenario Generation

Generate ensemble of compound hazard scenarios

In [ ]:
# Generate scenarios
scenarios = bn.sample_scenarios(
    n_scenarios=100,  # Small number for demo
    future_year=2050  # Project to 2050
)

print(f"Generated {len(scenarios)} compound hazard scenarios")
scenarios.head()

In [ ]:
# Visualize scenario distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, hazard in enumerate(hazard_types):
    if hazard in scenarios.columns:
        axes[i].hist(scenarios[hazard], bins=30, alpha=0.7, edgecolor='black')
        axes[i].set_xlabel(hazard.replace('_', ' ').title())
        axes[i].set_ylabel('Frequency')
        axes[i].set_title(f'{hazard.replace("_", " ").title()} Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate compound event probability
thresholds = {
    'storm_surge': 0.5,  # 0.5m
    'sea_level': 100,     # 100mm
    'precipitation': 50   # 50mm
}

compound_prob = bn.get_compound_probability(thresholds)
print(f"\nProbability of compound event exceeding thresholds: {compound_prob:.3f}")
print(f"Return period: ~{1/compound_prob:.1f} scenarios")

## 4. Agent-Based Vulnerability Model

Simulate household-level impacts and adaptation

In [ ]:
# Initialize ABM
abm = VulnerabilityModel(
    city='Lagos',
    n_households=1000,  # Small number for demo
    spatial_resolution=100,  # 100m grid cells
    width=50,
    height=50,
    seed=42
)

print(f"Initialized ABM with {abm.n_households} household agents")

In [ ]:
# Run simulation for scenario subset
print("Running agent-based simulation...")

results = abm.run(
    scenarios=scenarios.head(10),  # First 10 scenarios for demo
    n_steps=30  # 30 days per scenario
)

print("\nSimulation complete!")
results.head()

In [ ]:
# Visualize results over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot by scenario
for scenario_id in results['scenario_id'].unique()[:3]:  # First 3 scenarios
    scenario_data = results[results['scenario_id'] == scenario_id]

    axes[0, 0].plot(scenario_data.index, scenario_data['Total_Damage'], label=f'Scenario {scenario_id}')
    axes[0, 1].plot(scenario_data.index, scenario_data['Displaced_Households'], label=f'Scenario {scenario_id}')
    axes[1, 0].plot(scenario_data.index, scenario_data['Adaptation_Rate'], label=f'Scenario {scenario_id}')
    axes[1, 1].plot(scenario_data.index, scenario_data['Average_Risk_Perception'], label=f'Scenario {scenario_id}')

axes[0, 0].set_title('Total Economic Damage')
axes[0, 1].set_title('Displaced Households')
axes[1, 0].set_title('Adaptation Rate')
axes[1, 1].set_title('Average Risk Perception')

for ax in axes.flat:
    ax.legend()
    ax.set_xlabel('Time Step')

plt.tight_layout()
plt.show()

In [ ]:
# Generate vulnerability map
vulnerability_map = abm.get_spatial_vulnerability_map()

plt.figure(figsize=(10, 8))
plt.imshow(vulnerability_map, cmap='YlOrRd', origin='lower')
plt.colorbar(label='Vulnerability Score')
plt.title('Spatial Vulnerability Distribution')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.show()

## 5. Deep Uncertainty Analysis

Identify robust adaptation strategies

In [ ]:
# Initialize uncertainty analysis
rdm = DeepUncertaintyAnalysis(
    n_scenarios=100,
    objectives=['minimize_casualties', 'minimize_economic_loss']
)

# Define uncertain factors
uncertain_factors = {
    'slr_rate': (3, 10),           # mm/year
    'storm_frequency': (2, 6),     # events/year
    'population_growth': (0.02, 0.05),  # annual rate
    'adaptation_cost_multiplier': (0.5, 2.0)
}

# Generate uncertainty space
uncertainty_scenarios = rdm.generate_uncertainty_space(uncertain_factors)
print(f"Generated {len(uncertainty_scenarios)} scenarios in uncertainty space")
uncertainty_scenarios.head()

In [ ]:
# Define adaptation strategies
strategies = [
    {
        'name': 'Seawall',
        'cost': 50000000,  # $50M
        'protection_level': 0.8,
        'implementation_time': 3
    },
    {
        'name': 'Nature-based Solutions',
        'cost': 20000000,  # $20M
        'protection_level': 0.5,
        'implementation_time': 2
    },
    {
        'name': 'Managed Retreat',
        'cost': 30000000,  # $30M
        'protection_level': 1.0,  # Full protection (moved away)
        'implementation_time': 5
    },
    {
        'name': 'No Action',
        'cost': 0,
        'protection_level': 0.0,
        'implementation_time': 0
    }
]

for strategy in strategies:
    rdm.define_adaptation_strategy(
        name=strategy['name'],
        actions=[{'protection_level': strategy['protection_level']}],
        cost=strategy['cost'],
        implementation_time=strategy['implementation_time']
    )

print(f"Defined {len(strategies)} adaptation strategies")

## 6. Results and Recommendations

Summary of key findings

In [ ]:
# Summary statistics
print("="*60)
print("COASTAL RISK ASSESSMENT SUMMARY - LAGOS")
print("="*60)

print(f"\n1. DATA QUALITY")
print(f"   Overall Score: {processor.data_quality_score:.2f}/1.00")
print(f"   Recommendation: {processor._recommend_approach()}")

print(f"\n2. HAZARD SCENARIOS")
print(f"   Total scenarios generated: {len(scenarios)}")
print(f"   Compound event probability: {compound_prob:.3f}")
print(f"   Mean sea level (2050): {scenarios['sea_level'].mean():.1f} mm")
print(f"   Mean storm surge: {scenarios['storm_surge'].mean():.2f} m")

print(f"\n3. VULNERABILITY ASSESSMENT")
print(f"   Households simulated: {abm.n_households}")
print(f"   Average adaptation rate: {results['Adaptation_Rate'].mean():.2%}")
print(f"   Average risk perception: {results['Average_Risk_Perception'].mean():.2f}")

print(f"\n4. ADAPTATION STRATEGIES")
for strategy in strategies:
    print(f"   - {strategy['name']}: ${strategy['cost']:,.0f}, Protection: {strategy['protection_level']:.0%}")

print(f"\n5. KEY RECOMMENDATIONS")
print(f"   - Implement multi-layered adaptation approach")
print(f"   - Prioritize nature-based solutions for cost-effectiveness")
print(f"   - Enhance community risk awareness and preparedness")
print(f"   - Monitor sea level trends for adaptive management")

print("\n" + "="*60)

## Next Steps

1. **Refine Analysis**: Run with more scenarios and longer simulation periods
2. **Validate Results**: Compare with historical flood events
3. **Stakeholder Engagement**: Present findings to city planners
4. **Expand Coverage**: Apply to other coastal cities
5. **Real-time Integration**: Connect to operational early warning systems

For detailed documentation, see:
- `docs/methodology.md`: Complete methodology
- `docs/data_requirements.md`: Data specifications
- API documentation: Generated using Sphinx